# vectorRetrieval实现
> By SherryAGI | 语义搜索与混合Retrieval

## vectorRetrieval vs keywordRetrieval

| 特性 | keywordRetrieval | vectorRetrieval |
|------|------------|----------|
| 原理 | Exact match | Semantic similarity |
| Synonyms | ❌ Cannot recognize | ✅ Auto understand |
| Speed | ⚡ Very fast | 🔄 Fast |
| Use case | Exact search | Fuzzy search |

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

print('🚀 vectorRetrievalmodule by SherryAGI')

## 1. Embedding Models选择

In [ ]:
# recommended的 Embedding Models
embedding_models = {
    "Multilingual": [
        "BAAI/bge-m3",
        "intfloat/multilingual-e5-large"
    ],
    "Chinese": [
        "BAAI/bge-large-zh-v1.5",
        "shibing624/text2vec-base-chinese"
    ],
    "English": [
        "BAAI/bge-large-en-v1.5",
        "sentence-transformers/all-MiniLM-L6-v2"
    ],
    "API": [
        "OpenAI text-embedding-3-small",
        "Cohere embed-multilingual-v3.0"
    ]
}

print('📊 recommended Embedding Models:')
for category, models in embedding_models.items():
    print(f'\n{category}:')
    for m in models:
        print(f'  - {m}')

## 2. vectordata库实现

In [ ]:
class SimpleVectorDB:
    """简单vectordata库 by SherryAGI"""
    
    def __init__(self, embedding_dim=384):
        self.documents = []
        self.embeddings = []
        self.dim = embedding_dim
    
    def _mock_embed(self, text):
        """模拟Vectorization"""
        np.random.seed(hash(text) % 2**32)
        return np.random.rand(self.dim)
    
    def add(self, text, metadata=None):
        """Add document"""
        embedding = self._mock_embed(text)
        self.documents.append({"text": text, "metadata": metadata or {}})
        self.embeddings.append(embedding)
        return len(self.documents) - 1
    
    def search(self, query, top_k=5):
        """vector搜索"""
        query_embedding = self._mock_embed(query)
        
        similarities = cosine_similarity(
            [query_embedding], 
            self.embeddings
        )[0]
        
        # 排序
        indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in indices:
            results.append({
                "score": float(similarities[idx]),
                "text": self.documents[idx]["text"],
                "metadata": self.documents[idx]["metadata"]
            })
        
        return results

print('✅ SimpleVectorDB defined')

## 3. TestvectorRetrieval

In [ ]:
# 创建data库
db = SimpleVectorDB()

# 添加enterpriseknowledge
docs = [
    ("Reimbursement process：submit invoice，fill reimbursement form，主管approval", {"category": "Finance"}),
    ("Leave application：loginOAsystem，select leave type，提交approval", {"category": "HR"}),
    ("Meeting room booking：通过enterprise微信预订，up to7days", {"category": "Admin"}),
    ("VPNConnection：下载customer端，enter credentials，select server", {"category": "IT"}),
    ("Password reset：contactIThotline8888or self-service reset", {"category": "IT"}),
]

for text, meta in docs:
    db.add(text, meta)

print(f'✅ added {len(docs)} records')

In [ ]:
# Test搜索
queries = [
    "How to reimburse？",
    "Forgot password",
    "Want to take leave"
]

for query in queries:
    print(f'\n🔍 Query: {query}')
    results = db.search(query, top_k=2)
    for i, r in enumerate(results):
        print(f'  {i+1}. [{r["metadata"]["category"]}] {r["text"][:40]}... (Similarity: {r["score"]:.3f})')

## 4. 混合Retrieval

In [ ]:
class HybridSearch:
    """混合Retrieval：vector + keyword by SherryAGI"""
    
    def __init__(self):
        self.vector_db = SimpleVectorDB()
        self.keyword_index = {}  # 简单倒排索引
    
    def add(self, text, metadata=None):
        idx = self.vector_db.add(text, metadata)
        # 建立keyword索引
        for word in text.split():
            if word not in self.keyword_index:
                self.keyword_index[word] = []
            self.keyword_index[word].append(idx)
        return idx
    
    def search(self, query, top_k=5, alpha=0.7):
        """混合搜索 (alpha: vector权重)"""
        # vector搜索
        vector_results = self.vector_db.search(query, top_k * 2)
        
        # keyword搜索
        keyword_scores = {}
        for word in query.split():
            if word in self.keyword_index:
                for idx in self.keyword_index[word]:
                    keyword_scores[idx] = keyword_scores.get(idx, 0) + 1
        
        # 融合分数
        final_scores = {}
        for r in vector_results:
            idx = self.vector_db.documents.index(
                {"text": r["text"], "metadata": r["metadata"]}
            )
            vector_score = r["score"]
            keyword_score = keyword_scores.get(idx, 0) / max(len(query.split()), 1)
            final_scores[idx] = alpha * vector_score + (1 - alpha) * keyword_score
        
        # 排序返回
        sorted_indices = sorted(final_scores.keys(), key=lambda x: -final_scores[x])
        
        results = []
        for idx in sorted_indices[:top_k]:
            doc = self.vector_db.documents[idx]
            results.append({
                "score": final_scores[idx],
                "text": doc["text"],
                "metadata": doc["metadata"]
            })
        
        return results

print('✅ HybridSearch defined')

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)